In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
#np.random.seed(MASTER_SEED)
import tensorflow as tf
#tf.set_random_seed(MASTER_SEED)
import matplotlib.pyplot as plt
import keras

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
keras.backend.set_session(tf.compat.v1.Session(config=config))
    
import time
from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.layers import *
from keras.activations import relu, sigmoid
from keras.optimizers import Adam, Nadam

import cv2
import math

Using TensorFlow backend.


In [2]:
optimizer = Nadam(0.00007, 0.5) # 0.0002, 0.5
img_shape = (108, 88, 3)
latent_dim = 100

In [3]:
def build_discriminator():
    img_input = Input(shape=img_shape)
    #num_input = Input(shape=(10,))
    
    imgmodel = Sequential()
    imgmodel.add(Conv2D(64, kernel_size=8, strides=2, input_shape=img_shape, padding="same"))
    imgmodel.add(BatchNormalization(momentum=0.8))
    imgmodel.add(LeakyReLU(alpha=0.2))
    imgmodel.add(Conv2D(64, kernel_size=8, strides=2, padding="same"))
    imgmodel.add(BatchNormalization(momentum=0.8))
    imgmodel.add(LeakyReLU(alpha=0.2))
    imgmodel.add(Conv2D(32, kernel_size=16, padding="same"))
    imgmodel.add(BatchNormalization(momentum=0.8))
    imgmodel.add(LeakyReLU(alpha=0.2))
    imgmodel.add(Dropout(0.2))
    imgmodel.add(Flatten())
    imgmodel.add(Dense(16, activation="relu"))
    imgmodel.add(Dropout(0.1))
    imgmodel.add(Dense(1, activation="sigmoid"))
    imgmodel.summary()
    
    #imgout = imgmodel(img_input)
    #catout = Concatenate(axis=-1)([imgout, num_input]) 
    #finmodel = Sequential()
    #finmodel.add(Dropout(0.1))
    #finmodel.add(Dense(65, activation="relu", input_shape=[128,]))
    
    validity = imgmodel(img_input)
    
    model = Model(inputs=img_input, outputs=validity)
    return model

In [4]:
def build_generator():
    model = Sequential()

    model.add(Dense(27 * 22 * 8, input_shape=(latent_dim,)))
    model.add(Reshape((27, 22, 8)))
    model.add(Conv2D(128, kernel_size=16, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))
    model.add(UpSampling2D())
    model.add(Conv2D(128, kernel_size=8, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))
    model.add(UpSampling2D())
    model.add(Conv2D(64, kernel_size=4, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))
    model.add(Conv2D(16, kernel_size=4, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))
    model.add(Conv2D(3, kernel_size=2, padding="same"))
    model.add(Activation("tanh"))

    model.summary()

    noise = Input(shape=(latent_dim,))
    img = model(noise)

    return Model(noise, img)

In [5]:
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

generator = build_generator()

z = Input(shape=(100,))
img = generator(z)
discriminator.trainable = False
valid = discriminator(img)

combined = Model(z, valid)
combined.summary()
combined.compile(loss='binary_crossentropy', optimizer=optimizer)







Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 54, 44, 64)        12352     
_________________________________________________________________
batch_normalization_1 (Batch (None, 54, 44, 64)        256       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 54, 44, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 27, 22, 64)        262208    
_________________________________________________________________
batch_normalization_2 (Batch (None, 27, 22, 64)        256       
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 27, 22, 64)        0         
_____________________

In [6]:
if 'imgs' in globals():
    del imgs
imgnum = 50000 # 202599
X_train = np.empty([imgnum, 108, 88, 3], dtype=np.uint8)
interv = int(math.ceil(imgnum/10))
for i in range(imgnum):
    if(i%interv==0):
        print(int(i//interv), end="")
    X_train[i] = cv2.cvtColor(cv2.imread("img_mod/%06d.jpg" % (i+1)), cv2.COLOR_BGR2RGB)
X_train = (X_train / 127.5) - 0.5
print("\nDone loading.")

#X_train = np.expand_dims(X_train, )

0123456789
Done loading.


In [7]:
def train(epochs, batch_size=128, print_interval=10, save_interval=100):
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    pc = time.perf_counter()
    start_time = pc
    for epoch in range(epochs):
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        imgs = X_train[idx]

        lspace = np.random.normal(0, 1, (batch_size, latent_dim))
        gen_imgs = generator.predict(lspace)
        
        d_loss_real = discriminator.train_on_batch(imgs, valid) # TODO (maybe): give the fake images a different set of labels
        d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        
        g_loss = combined.train_on_batch(lspace, valid)

        if epoch%print_interval==0:
            td = time.perf_counter()-pc
            pc = time.perf_counter()
            eta = (pc-start_time) * (epochs - (epoch+1)) / (epoch+1)
            etas = eta%60
            eta //= 60
            etam = eta%60
            eta //= 60
            print ("%d [dl: %.3f, acc: %.2f%%] [gl: %f] - [tpe: %.2f, eta: %d:%02d:%02d]" % \
                   (epoch, d_loss[0], 100*d_loss[1], g_loss, td/print_interval, eta, etam, etas))
        
        if epoch%save_interval==0:
            r, c = 2, 4
            lspace = np.random.normal(0, 1, (r * c, latent_dim))
            gen_imgs = generator.predict(lspace)
            
            gen_imgs = 0.5 * gen_imgs + 0.5

            fig, axs = plt.subplots(r, c)
            cnt = 0
            for i in range(r):
                for j in range(c):
                    axs[i,j].imshow(gen_imgs[cnt])
                    axs[i,j].axis('off')
                    cnt += 1
            fig.savefig("out_img/ep%05d.png" % epoch)
            plt.close()

In [8]:
train(20000, batch_size=64, print_interval=10, save_interval=10)

0 [dl: 1.050, acc: 23.44%] [gl: 1.304974] - [tpe: 0.78, eta: 43:03:47]
10 [dl: 0.149, acc: 96.09%] [gl: 3.364233] - [tpe: 0.37, eta: 5:46:23]
20 [dl: 0.095, acc: 98.44%] [gl: 9.020041] - [tpe: 0.33, eta: 3:53:39]
30 [dl: 0.008, acc: 100.00%] [gl: 0.409448] - [tpe: 0.33, eta: 3:13:24]
40 [dl: 0.534, acc: 73.44%] [gl: 16.098286] - [tpe: 0.33, eta: 2:53:03]
50 [dl: 0.040, acc: 100.00%] [gl: 0.736079] - [tpe: 0.33, eta: 2:40:18]
60 [dl: 0.118, acc: 98.44%] [gl: 1.010957] - [tpe: 0.32, eta: 2:31:34]
70 [dl: 0.040, acc: 99.22%] [gl: 0.808441] - [tpe: 0.33, eta: 2:25:36]
80 [dl: 0.027, acc: 100.00%] [gl: 2.301273] - [tpe: 0.33, eta: 2:21:16]
90 [dl: 0.019, acc: 99.22%] [gl: 0.214146] - [tpe: 0.34, eta: 2:18:09]
100 [dl: 0.028, acc: 99.22%] [gl: 2.228600] - [tpe: 0.32, eta: 2:15:02]
110 [dl: 0.200, acc: 96.09%] [gl: 4.476274] - [tpe: 0.32, eta: 2:12:25]
120 [dl: 0.085, acc: 99.22%] [gl: 2.679534] - [tpe: 0.33, eta: 2:10:19]
130 [dl: 0.170, acc: 96.88%] [gl: 4.217381] - [tpe: 0.32, eta: 2:08:29

In [9]:
for pic in range(10):
    r, c = 2, 4
    lspace = np.random.normal(0, 1, (r * c, latent_dim))
    gen_imgs = generator.predict(lspace)

    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt])
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("out_img/FINAL%d.png" % pic)
    plt.close()